## परिचय

यो पाठमा समेटिनेछ:
- फंक्शन कल के हो र यसको प्रयोगका क्षेत्रहरू
- Azure OpenAI प्रयोग गरेर फंक्शन कल कसरी बनाउने
- फंक्शन कललाई एप्लिकेसनमा कसरी एकीकृत गर्ने

## सिकाइका उद्देश्यहरू

यो पाठ पूरा गरेपछि तपाईंले जान्नुहुनेछ र बुझ्नुहुनेछ:

- फंक्शन कल प्रयोग गर्ने उद्देश्य
- Azure Open AI Service प्रयोग गरेर फंक्शन कल सेटअप गर्ने तरिका
- तपाईंको एप्लिकेसनको प्रयोगका लागि प्रभावकारी फंक्शन कल डिजाइन गर्ने तरिका


## फंक्शन कलहरू बुझ्दै

यस पाठका लागि, हामी हाम्रो शिक्षा स्टार्टअपका लागि एउटा फिचर बनाउन चाहन्छौं जसले प्रयोगकर्ताहरूलाई च्याटबोट प्रयोग गरेर प्राविधिक कोर्सहरू खोज्न सहयोग गर्छ। हामी तिनीहरूको सीप स्तर, हालको भूमिका र चासोको प्रविधिमा आधारित कोर्सहरू सिफारिस गर्नेछौं।

यसलाई पूरा गर्न हामीले यी कुराहरूको संयोजन प्रयोग गर्नेछौं:
 - `Azure Open AI` प्रयोगकर्ताका लागि च्याट अनुभव बनाउन
 - `Microsoft Learn Catalog API` प्रयोगकर्ताको अनुरोध अनुसार कोर्सहरू खोज्न सहयोग गर्न
 - `Function Calling` प्रयोगकर्ताको सोधाइ लिएर त्यसलाई फंक्शनमा पठाएर API अनुरोध गर्न

सुरु गर्न, हामीले पहिलोपटक फंक्शन कल किन प्रयोग गर्ने भन्ने कुरा बुझौं:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPT बाट नयाँ प्रतिक्रिया प्राप्त गर्नुहोस् जहाँ उसले फंक्शनको प्रतिक्रिया देख्न सक्छ


print(second_response.choices[0].message)


### किन फंक्शन कलिंग

यदि तपाईंले यस कोर्सको कुनै अन्य पाठ पूरा गर्नुभएको छ भने, तपाईंले सम्भवतः ठूलो भाषा मोडेल (LLMs) प्रयोग गर्दा आउने फाइदाहरू बुझ्नुभएको छ। आशा छ, तपाईंले तिनीहरूको केही सीमाहरू पनि देख्नुभएको छ।

फंक्शन कलिंग भनेको Azure Open AI Service को एउटा सुविधा हो, जसले निम्न सीमाहरूलाई पार गर्न मद्दत गर्छ:
१) जवाफको ढाँचा सधैं एउटै राख्न सकिने
२) च्याट सन्दर्भमा एप्लिकेसनका अन्य स्रोतबाट डेटा प्रयोग गर्न सकिने क्षमता

फंक्शन कलिंग आउनु अघि, LLM बाट आउने जवाफहरू असंगठित र असमान हुन्थे। विकासकर्ताहरूले प्रत्येक फरक जवाफलाई सम्हाल्नका लागि जटिल प्रमाणीकरण कोड लेख्नुपर्ने हुन्थ्यो।

प्रयोगकर्ताहरूले "स्टकहोमको अहिलेको मौसम कस्तो छ?" जस्ता प्रश्नको जवाफ पाउन सक्दैनथे। किनभने मोडेलहरूमा प्रयोग भएको डेटा प्रशिक्षणको समयमा सीमित हुन्थ्यो।

अब, तलको उदाहरण हेर्नुहोस्, जसले यो समस्या देखाउँछ:

मानौं हामी विद्यार्थीहरूको डेटा राख्ने डाटाबेस बनाउन चाहन्छौं, ताकि उनीहरूलाई उपयुक्त कोर्स सिफारिस गर्न सकियोस्। तल दुई विद्यार्थीको विवरण छ, जसमा भएको जानकारी धेरै मिल्दोजुल्दो छ।


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

हामी यो डाटा पार्स गर्न LLM मा पठाउन चाहन्छौं। यो पछि हाम्रो एप्लिकेशनमा API मा पठाउन वा डाटाबेसमा भण्डारण गर्न प्रयोग गर्न सकिन्छ।

हामी दुईवटा उस्तै प्रॉम्प्टहरू बनाउँछौं जसमा हामी LLM लाई कुन जानकारीमा चासो छ भनेर निर्देशन दिन्छौं:


हामी यो हाम्रो उत्पादनका लागि महत्त्वपूर्ण भागहरू पार्स गर्न LLM लाई पठाउन चाहन्छौं। त्यसैले हामी LLM लाई निर्देशन दिन दुईवटा उस्तै प्रॉम्प्टहरू बनाउन सक्छौं:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


यी दुई प्रॉम्प्टहरू सिर्जना गरेपछि, हामी तिनीहरूलाई LLM मा `openai.ChatCompletion` प्रयोग गरेर पठाउनेछौं। हामी प्रॉम्प्टलाई `messages` भेरिएबलमा भण्डारण गर्छौं र भूमिका `user` मा तोक्छौं। यो प्रयोगकर्ताबाट च्याटबोटमा लेखिएको सन्देशको नक्कल गर्नका लागि हो।


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

यद्यपि प्रॉम्प्टहरू उस्तै छन् र विवरणहरू पनि मिल्दोजुल्दो छन्, हामीले `Grades` प्रोपर्टीका विभिन्न स्वरूपहरू पाउन सक्छौं।

यदि तपाईं माथिको सेल धेरै पटक चलाउनु भयो भने, स्वरूप `3.7` वा `3.7 GPA` हुन सक्छ।

यो किनभने LLM ले लेखिएको प्रॉम्प्टको रूपमा असंरचित डेटा लिन्छ र असंरचित डेटा नै फर्काउँछ। हामीलाई संरचित स्वरूप चाहिन्छ ताकि यो डेटा भण्डारण गर्दा वा प्रयोग गर्दा के अपेक्षा गर्ने भन्ने थाहा होस्।

फंक्शनल कलिङ प्रयोग गर्दा, हामीले संरचित डेटा फिर्ता पाउँछौं भन्ने सुनिश्चित गर्न सक्छौं। फंक्शन कलिङ प्रयोग गर्दा, LLM ले वास्तवमा कुनै फंक्शन चलाउँदैन वा कल गर्दैन। बरु, हामीले LLM लाई यसको प्रतिक्रिया दिनका लागि एउटा संरचना बनाउँछौं। त्यसपछि ती संरचित प्रतिक्रियाहरू प्रयोग गरेर हाम्रो एप्लिकेसनमा कुन फंक्शन चलाउने भन्ने थाहा पाउँछौं।


![फंक्शन कलिङ प्रवाह चित्र](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ne.png)


### फंक्शन कल प्रयोग गर्ने उपयोगका केसहरू

**बाह्य उपकरणहरू कल गर्नु**
च्याटबोटहरू प्रयोगकर्ताबाट आएका प्रश्नहरूको उत्तर दिनमा निकै सक्षम छन्। फंक्शन कलको प्रयोग गरेर, च्याटबोटहरूले प्रयोगकर्ताका सन्देशहरू प्रयोग गरेर केही कार्यहरू पूरा गर्न सक्छन्। उदाहरणका लागि, विद्यार्थीले च्याटबोटलाई "मेरो शिक्षकलाई इमेल पठाइदिनुहोस् कि मलाई यो विषयमा थप सहयोग चाहिन्छ" भन्न सक्छ। यसले `send_email(to: string, body: string)` नामको फंक्शन कल गर्न सक्छ।

**API वा डाटाबेस क्वेरीहरू बनाउनु**
प्रयोगकर्ताहरूले प्राकृतिक भाषामा जानकारी खोज्न सक्छन् जुन फर्म्याट गरिएको क्वेरी वा API अनुरोधमा रूपान्तरण हुन्छ। यसको उदाहरणका रूपमा, शिक्षकले "पछिल्लो असाइनमेन्ट पूरा गरेका विद्यार्थीहरू को हुन्?" भनेर सोध्न सक्छन्, जसले `get_completed(student_name: string, assignment: int, current_status: string)` नामको फंक्शन कल गर्न सक्छ।

**संरचित डाटा बनाउनु**
प्रयोगकर्ताहरूले कुनै पाठको ब्लक वा CSV लिएर LLM प्रयोग गरेर त्यसबाट महत्वपूर्ण जानकारी निकाल्न सक्छन्। उदाहरणका लागि, विद्यार्थीले शान्ति सम्झौताबारेको विकिपिडिया लेखलाई AI फ्ल्यास कार्ड बनाउन रूपान्तरण गर्न सक्छ। यो `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` नामको फंक्शन प्रयोग गरेर गर्न सकिन्छ।


## २. आफ्नो पहिलो फंक्शन कल बनाउने

फंक्शन कल बनाउने प्रक्रिया तीन मुख्य चरणहरूमा विभाजित छ:
१. तपाईंका फंक्शनहरूको सूची र प्रयोगकर्ताको सन्देशसहित Chat Completions API मा कल गर्नुहोस्
२. मोडेलको प्रतिक्रिया पढ्नुहोस् र आवश्यक कार्य गर्नुहोस्, जस्तै फंक्शन वा API कल चलाउनु
३. तपाईंको फंक्शनबाट आएको प्रतिक्रियासहित फेरि Chat Completions API मा कल गर्नुहोस्, ताकि त्यो जानकारी प्रयोग गरेर प्रयोगकर्तालाई जवाफ तयार गर्न सकियोस्।


![फङ्सन कलको प्रवाह](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ne.png)


### फंक्शन कलका तत्वहरू

#### प्रयोगकर्ताको इनपुट

पहिलो चरण भनेको प्रयोगकर्ताको सन्देश तयार गर्नु हो। यो सन्देशलाई तपाईंले टेक्स्ट इनपुटको मान लिएर गतिशील रूपमा सेट गर्न सक्नुहुन्छ, वा यहाँ नै मान दिन सक्नुहुन्छ। यदि तपाईं पहिलो पटक Chat Completions API सँग काम गर्दै हुनुहुन्छ भने, हामीले सन्देशको `role` र `content` परिभाषित गर्नुपर्छ।

`role` तीन मध्ये कुनै एक हुन सक्छ: `system` (नियम बनाउने), `assistant` (मोडेल), वा `user` (अन्तिम प्रयोगकर्ता)। फंक्शन कलका लागि, हामी यसलाई `user` राख्नेछौं र एउटा उदाहरण प्रश्न दिनेछौं।


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### फंक्शनहरू बनाउँदै।

अब हामी एउटा फंक्शन र त्यस फंक्शनका प्यारामिटरहरू परिभाषित गर्नेछौं। यहाँ हामी `search_courses` नामको एउटा मात्र फंक्शन प्रयोग गर्नेछौं, तर तपाईंले धेरै फंक्शनहरू पनि बनाउन सक्नुहुन्छ।

**महत्वपूर्ण** : फंक्शनहरू LLM को सिस्टम सन्देशमा समावेश गरिन्छन् र तपाईंले प्रयोग गर्न सक्ने उपलब्ध टोकनको संख्यामा पनि गनिन्छ।


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**परिभाषाहरू**

`name` -  त्यो फंक्शनको नाम जुन हामीले कल गर्न चाहेका छौं।

`description` - यो फंक्शन कसरी काम गर्छ भन्ने बारेको वर्णन हो। यहाँ स्पष्ट र विशेष हुनु महत्त्वपूर्ण छ।

`parameters` - मोडेलले आफ्नो उत्तरमा उत्पादन गर्नुपर्ने मानहरू र ढाँचाहरूको सूची

`type` -  गुणहरूको डेटा प्रकार जसमा भण्डारण गरिनेछ

`properties` - मोडेलले आफ्नो उत्तरका लागि प्रयोग गर्ने विशेष मानहरूको सूची

`name` - मोडेलले आफ्नो ढाँचाबद्ध उत्तरमा प्रयोग गर्ने गुणको नाम

`type` - यो गुणको डेटा प्रकार

`description` - विशेष गुणको वर्णन

**वैकल्पिक**

`required` - फंक्शन कल पूरा गर्न आवश्यक गुण


### फंक्शन कल गर्ने तरिका
फंक्शन परिभाषित गरेपछि, अब हामीले यसलाई Chat Completion API को कलमा समावेश गर्नुपर्छ। यो गर्नका लागि, हामीले अनुरोधमा `functions` थप्नुपर्छ। यस अवस्थामा `functions=functions` हुन्छ।

यहाँ `function_call` लाई `auto` मा सेट गर्ने विकल्प पनि छ। यसको अर्थ, प्रयोगकर्ताको सन्देशको आधारमा कुन फंक्शन कल गर्ने भन्ने निर्णय LLM ले आफैं गर्छ, हामीले छुट्टै तोक्नुपर्दैन।


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

अब हामी प्रतिक्रिया हेर्छौं र यसको ढाँचा कस्तो छ भनेर बुझ्छौं:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

यहाँ तपाईं देख्न सक्नुहुन्छ कि function को नाम बोलाइएको छ र प्रयोगकर्ताको सन्देशबाट, LLM ले function का arguments मा मिल्ने डेटा फेला पार्न सकेको छ।


## ३. एप्लिकेसनमा फंक्शन कलहरू एकीकृत गर्दै

हामीले LLM बाट आएको फर्म्याट गरिएको प्रतिक्रिया परीक्षण गरेपछि, अब यसलाई एप्लिकेसनमा एकीकृत गर्न सक्छौं।

### फ्लो व्यवस्थापन गर्दै

यसलाई हाम्रो एप्लिकेसनमा एकीकृत गर्नका लागि, तलका चरणहरू अपनाऔं:

पहिले, Open AI सेवामा कल गरौं र सन्देशलाई `response_message` नामको भेरिएबलमा राखौं।


In [ ]:
response_message = response.choices[0].message

अब हामीले त्यो फंक्शन परिभाषित गर्नेछौं जसले Microsoft Learn API लाई कल गरेर कोर्सहरूको सूची ल्याउनेछ:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



एक राम्रो अभ्यासको रूपमा, हामी त्यसपछि हेर्नेछौं कि मोडेलले कुनै फंक्शन कल गर्न चाहन्छ कि चाहँदैन। त्यसपछि, हामी उपलब्ध फंक्शनहरू मध्ये एक बनाउनेछौं र त्यसलाई कल गर्न लागिएको फंक्शनसँग मिलाउनेछौं।

त्यसपछि, हामी फंक्शनका आर्गुमेन्टहरू लिनेछौं र तिनीहरूलाई LLM बाट आएका आर्गुमेन्टहरूसँग मिलाउनेछौं।

अन्त्यमा, हामी फंक्शन कल सन्देश र `search_courses` सन्देशबाट फर्किएका मानहरू थप्नेछौं। यसले LLM लाई प्रयोगकर्तालाई प्राकृतिक भाषामा जवाफ दिन आवश्यक सबै जानकारी दिन्छ।


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## कोड चुनौती

शानदार काम! Azure Open AI Function Calling को सिकाइलाई अगाडि बढाउन तपाईंले यो बनाउन सक्नुहुन्छ: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - फंक्शनका थप प्यारामिटरहरू जसले सिक्नेहरूलाई थप कोर्सहरू फेला पार्न मद्दत गर्न सक्छ। तपाईं यहाँ उपलब्ध API प्यारामिटरहरू फेला पार्न सक्नुहुन्छ:  
 - अर्को फंक्शन कल बनाउनुहोस् जसले सिक्नेको मातृभाषा जस्ता थप जानकारी लिन्छ  
 - जब फंक्शन कल र/वा API कलले उपयुक्त कोर्सहरू फिर्ता गर्दैन, त्यस्तो अवस्थामा त्रुटि ह्यान्डलिङ बनाउनुहोस्



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल भाषा मा रहेको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीको लागि, पेशेवर मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
